In [16]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import inception_v3
import cv2
import os
import numpy as np
from PIL import Image

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the deepfake detector model
class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.facenet = inception_v3(pretrained=False, aux_logits=False)
        in_features = self.facenet.fc.in_features
        self.facenet.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.facenet(x)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize model
model = DeepFakeDetector().to(device)

# Load trained weights
model.load_state_dict(torch.load("deepfake_detector.pth", map_location=device))
model.eval()

# Function to extract frames from video
def extract_frames(video_path, frame_rate=1):
    """
    Extracts frames from a video at a specified frame rate.
    Args:
        video_path (str): Path to the video file.
        frame_rate (int): Number of frames to extract per second.
    Returns:
        List of frames as PIL images.
    """
    frames = []
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = fps // frame_rate

    count = 0
    success, frame = cap.read()
    while success:
        if count % frame_interval == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(frame))
        success, frame = cap.read()
        count += 1

    cap.release()
    return frames

# Function to classify frames
def classify_frames(frames):
    """
    Classify each frame as Real or Deepfake.
    Args:
        frames (list): List of PIL images (frames).
    Returns:
        List of classifications (Real/Deepfake) for each frame.
    """
    results = []
    for frame in frames:
        img = transform(frame).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(img).item()
            prediction = "Real" if output < 0.5 else "Deepfake"
            results.append(prediction)
    return results

# Function to process video and aggregate predictions
def process_video(video_path):
    """
    Processes a video, extracts frames, classifies them, and determines the overall classification.
    Args:
        video_path (str): Path to the video file.
    Returns:
        Dict with frame-level and video-level classifications.
    """
    frames = extract_frames(video_path)
    frame_predictions = classify_frames(frames)
    overall_prediction = "Real" if frame_predictions.count("Real") > frame_predictions.count("Deepfake") else "Deepfake"
    
    return {
        "video_path": video_path,
        "frame_predictions": frame_predictions,
        "overall_prediction": overall_prediction
    }

# Process videos in a directory
video_dir = "D:/editor/DFD_A/Real/03__kitchen_still.mp4"
results = []
for video_file in os.listdir(video_dir):
    if video_file.endswith((".mp4", ".avi", ".mov")):  # Add more formats as needed
        video_path = os.path.join(video_dir, video_file)
        result = process_video(video_path)
        results.append(result)

# Display results
for result in results:
    print(f"Video: {result['video_path']}")
    print(f"Overall Prediction: {result['overall_prediction']}")
    print(f"Frame Predictions: {result['frame_predictions']}")
    print("-" * 40)


C:\Users\Dilshan\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Dilshan\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Dilshan\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


RuntimeError: Error(s) in loading state_dict for DeepFakeDetector:
	Missing key(s) in state_dict: "facenet.Conv2d_1a_3x3.conv.weight", "facenet.Conv2d_1a_3x3.bn.weight", "facenet.Conv2d_1a_3x3.bn.bias", "facenet.Conv2d_1a_3x3.bn.running_mean", "facenet.Conv2d_1a_3x3.bn.running_var", "facenet.Conv2d_2a_3x3.conv.weight", "facenet.Conv2d_2a_3x3.bn.weight", "facenet.Conv2d_2a_3x3.bn.bias", "facenet.Conv2d_2a_3x3.bn.running_mean", "facenet.Conv2d_2a_3x3.bn.running_var", "facenet.Conv2d_2b_3x3.conv.weight", "facenet.Conv2d_2b_3x3.bn.weight", "facenet.Conv2d_2b_3x3.bn.bias", "facenet.Conv2d_2b_3x3.bn.running_mean", "facenet.Conv2d_2b_3x3.bn.running_var", "facenet.Conv2d_3b_1x1.conv.weight", "facenet.Conv2d_3b_1x1.bn.weight", "facenet.Conv2d_3b_1x1.bn.bias", "facenet.Conv2d_3b_1x1.bn.running_mean", "facenet.Conv2d_3b_1x1.bn.running_var", "facenet.Conv2d_4a_3x3.conv.weight", "facenet.Conv2d_4a_3x3.bn.weight", "facenet.Conv2d_4a_3x3.bn.bias", "facenet.Conv2d_4a_3x3.bn.running_mean", "facenet.Con